# Introduction
This notebook is loading and processing the data from the kaggle movie dataset

https://www.kaggle.com/rounakbanik/the-movies-dataset

in order to generate a minimalistic movie database table, and two dictionaries that contain keywords and characters for each movie genre.

In [1]:
import numpy as np
import pandas as pd
import ast
import itertools
import json
from matplotlib import pyplot as plt

In [2]:
ls data

credits.csv*         keywords.csv*
datatable.csv        movies_metadata.csv*


In [3]:
mm = pd.read_csv('data/movies_metadata.csv')
mm.head()

/Users/manuel.pasieka/anaconda3/envs/py3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
# Filter entries that cannot be parsed correctly
mm.drop(mm[~mm['id'].str.isnumeric()].index, inplace=True)

In [5]:
mm = mm.astype({'id': 'int'})

# Extract primary genre for each movie

In [6]:
# Extract primary genrey for each movie
mm['prim_genre'] = mm['genres'].apply(lambda x: ast.literal_eval(x)[0]['name'] if x != '[]' else '')
mm['prim_genre'].value_counts()

Drama              11966
Comedy              8820
Action              4489
Documentary         3415
Horror              2619
                    2442
Crime               1685
Thriller            1665
Adventure           1514
Romance             1191
Animation           1124
Fantasy              704
Science Fiction      647
Mystery              554
Family               524
Music                487
Western              451
TV Movie             390
War                  379
History              279
Foreign              118
Name: prim_genre, dtype: int64

In [7]:
movie_genre = mm[['prim_genre', 'id', 'original_title']]
print(movie_genre.dtypes)
movie_genre.head(10)

prim_genre        object
id                 int64
original_title    object
dtype: object


,prim_genre,id,original_title
0,Animation,862,Toy Story
1,Adventure,8844,Jumanji
2,Romance,15602,Grumpier Old Men
3,Comedy,31357,Waiting to Exhale
4,Comedy,11862,Father of the Bride Part II
5,Action,949,Heat
6,Comedy,11860,Sabrina
7,Action,45325,Tom and Huck
8,Action,9091,Sudden Death
9,Adventure,710,GoldenEye


# Load Keywords

In [8]:
kw = pd.read_csv('data/keywords.csv')
print(kw.dtypes)
kw.head(10)

id           int64
keywords    object
dtype: object


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
5,949,"[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
6,11860,"[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
7,45325,[]
8,9091,"[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."
9,710,"[{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam..."


In [9]:
kw['keyword_list'] = kw['keywords'].apply(lambda movie: [x['name'] for x in ast.literal_eval(movie)])
kw.drop('keywords', axis=1, inplace=True)
kw.head(10)

,id,keyword_list
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."
5,949,"[robbery, detective, bank, obsession, chase, s..."
6,11860,"[paris, brother brother relationship, chauffeu..."
7,45325,[]
8,9091,"[terrorist, hostage, explosive, vice president]"
9,710,"[cuba, falsely accused, secret identity, compu..."


# Generate a minimalistic movie db
Extract movie
* id
* keywords
* primary genre
* title

In [10]:
#movie_minimal.groupby('prim_genre').appl
min_datatable = pd.merge(kw, movie_genre, on='id')
min_datatable.head(10)

,id,keyword_list,prim_genre,original_title
0,862,"[jealousy, toy, boy, friendship, friends, riva...",Animation,Toy Story
1,8844,"[board game, disappearance, based on children'...",Adventure,Jumanji
2,15602,"[fishing, best friend, duringcreditsstinger, o...",Romance,Grumpier Old Men
3,31357,"[based on novel, interracial relationship, sin...",Comedy,Waiting to Exhale
4,11862,"[baby, midlife crisis, confidence, aging, daug...",Comedy,Father of the Bride Part II
5,949,"[robbery, detective, bank, obsession, chase, s...",Action,Heat
6,11860,"[paris, brother brother relationship, chauffeu...",Comedy,Sabrina
7,45325,[],Action,Tom and Huck
8,9091,"[terrorist, hostage, explosive, vice president]",Action,Sudden Death
9,710,"[cuba, falsely accused, secret identity, compu...",Adventure,GoldenEye


In [11]:
min_datatable.rename(columns={'keyword_list': 'keywords', 'prim_genre':'genre', 'original_title': 'title'}, inplace=True)
min_datatable.head()

,id,keywords,genre,title
0,862,"[jealousy, toy, boy, friendship, friends, riva...",Animation,Toy Story
1,8844,"[board game, disappearance, based on children'...",Adventure,Jumanji
2,15602,"[fishing, best friend, duringcreditsstinger, o...",Romance,Grumpier Old Men
3,31357,"[based on novel, interracial relationship, sin...",Comedy,Waiting to Exhale
4,11862,"[baby, midlife crisis, confidence, aging, daug...",Comedy,Father of the Bride Part II


In [12]:
# Filter entries with either no genre or no keywords
min_datatable = min_datatable[min_datatable['keywords'].apply(len) > 0]
min_datatable = min_datatable[min_datatable['genre'] != '']
min_datatable.head(10)

,id,keywords,genre,title
0,862,"[jealousy, toy, boy, friendship, friends, riva...",Animation,Toy Story
1,8844,"[board game, disappearance, based on children'...",Adventure,Jumanji
2,15602,"[fishing, best friend, duringcreditsstinger, o...",Romance,Grumpier Old Men
3,31357,"[based on novel, interracial relationship, sin...",Comedy,Waiting to Exhale
4,11862,"[baby, midlife crisis, confidence, aging, daug...",Comedy,Father of the Bride Part II
5,949,"[robbery, detective, bank, obsession, chase, s...",Action,Heat
6,11860,"[paris, brother brother relationship, chauffeu...",Comedy,Sabrina
8,9091,"[terrorist, hostage, explosive, vice president]",Action,Sudden Death
9,710,"[cuba, falsely accused, secret identity, compu...",Adventure,GoldenEye
10,9087,"[white house, usa president, new love, widower...",Comedy,The American President


In [13]:
# Store processed table results
mdb = min_datatable

# Generate keyword sets for each genre

In [14]:
G = mdb.groupby('genre')
# Genre keywords
gkw = {}
for genre, dt in G:
    # Needs to be list for the json serialization to work (set is not serializable)
    unique_kw = list(set(itertools.chain.from_iterable(dt['keywords'].values)))
    gkw[genre] = unique_kw
    print('%s: %d unique keywords' % (genre, len(unique_kw)))

Action: 5922 unique keywords
Adventure: 3583 unique keywords
Animation: 1878 unique keywords
Comedy: 7905 unique keywords
Crime: 3138 unique keywords
Documentary: 3246 unique keywords
Drama: 9854 unique keywords
Family: 780 unique keywords
Fantasy: 1773 unique keywords
Foreign: 150 unique keywords
History: 656 unique keywords
Horror: 3928 unique keywords
Music: 847 unique keywords
Mystery: 1186 unique keywords
Romance: 2116 unique keywords
Science Fiction: 1364 unique keywords
TV Movie: 570 unique keywords
Thriller: 2916 unique keywords
War: 627 unique keywords
Western: 704 unique keywords


# Generate character name sets for each genre

In [15]:
credits = pd.read_csv('data/credits.csv')
credits.head(10)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710


In [16]:
credits['characters'] = credits['cast'].apply(lambda movie: [x['character'] for x in ast.literal_eval(movie)])
credits.head(10)

,cast,crew,id,characters
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Woody (voice), Buzz Lightyear (voice), Mr. Po..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Alan Parrish, Samuel Alan Parrish / Van Pelt,..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[Max Goldman, John Gustafson, Ariel Gustafson,..."
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"[Savannah 'Vannah' Jackson, Bernadine 'Bernie'..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[George Banks, Nina Banks, Franck Eggelhoffer,..."
5,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...",949,"[Lt. Vincent Hanna, Neil McCauley, Chris Shihe..."
6,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...",11860,"[Linus Larrabee, Sabrina Fairchild, David Larr..."
7,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",45325,"[Tom Sawyer, Huck Finn, Becky Thatcher, Muff P..."
8,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...",9091,"[Darren Francis Thomas McCord, Joshua Foss, Ma..."
9,"[{'cast_id': 1, 'character': 'James Bond', 'cr...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de...",710,"[James Bond, Alec Trevelyan, Natalya Fyodorovn..."


In [17]:
# Filter again entries without characters
credits = credits[credits['characters'].apply(lambda x: len(x) > 0)]
credits.drop(['cast', 'crew'], axis=1, inplace=True)
credits.head(10)

,id,characters
0,862,"[Woody (voice), Buzz Lightyear (voice), Mr. Po..."
1,8844,"[Alan Parrish, Samuel Alan Parrish / Van Pelt,..."
2,15602,"[Max Goldman, John Gustafson, Ariel Gustafson,..."
3,31357,"[Savannah 'Vannah' Jackson, Bernadine 'Bernie'..."
4,11862,"[George Banks, Nina Banks, Franck Eggelhoffer,..."
5,949,"[Lt. Vincent Hanna, Neil McCauley, Chris Shihe..."
6,11860,"[Linus Larrabee, Sabrina Fairchild, David Larr..."
7,45325,"[Tom Sawyer, Huck Finn, Becky Thatcher, Muff P..."
8,9091,"[Darren Francis Thomas McCord, Joshua Foss, Ma..."
9,710,"[James Bond, Alec Trevelyan, Natalya Fyodorovn..."


In [18]:
mdb = pd.merge(mdb, credits, on='id')
mdb.head(10)

,id,keywords,genre,title,characters
0,862,"[jealousy, toy, boy, friendship, friends, riva...",Animation,Toy Story,"[Woody (voice), Buzz Lightyear (voice), Mr. Po..."
1,8844,"[board game, disappearance, based on children'...",Adventure,Jumanji,"[Alan Parrish, Samuel Alan Parrish / Van Pelt,..."
2,15602,"[fishing, best friend, duringcreditsstinger, o...",Romance,Grumpier Old Men,"[Max Goldman, John Gustafson, Ariel Gustafson,..."
3,31357,"[based on novel, interracial relationship, sin...",Comedy,Waiting to Exhale,"[Savannah 'Vannah' Jackson, Bernadine 'Bernie'..."
4,11862,"[baby, midlife crisis, confidence, aging, daug...",Comedy,Father of the Bride Part II,"[George Banks, Nina Banks, Franck Eggelhoffer,..."
5,949,"[robbery, detective, bank, obsession, chase, s...",Action,Heat,"[Lt. Vincent Hanna, Neil McCauley, Chris Shihe..."
6,11860,"[paris, brother brother relationship, chauffeu...",Comedy,Sabrina,"[Linus Larrabee, Sabrina Fairchild, David Larr..."
7,9091,"[terrorist, hostage, explosive, vice president]",Action,Sudden Death,"[Darren Francis Thomas McCord, Joshua Foss, Ma..."
8,710,"[cuba, falsely accused, secret identity, compu...",Adventure,GoldenEye,"[James Bond, Alec Trevelyan, Natalya Fyodorovn..."
9,9087,"[white house, usa president, new love, widower...",Comedy,The American President,"[Andrew Shepherd, Sydney Ellen Wade, Lewis Rot..."


In [19]:
G = mdb.groupby('genre')
# Genre characters
gch = {}
for genre, dt in G:
    # Needs to be list for the json serialization to work (set is not serializable)
    unique_characters = list(set(itertools.chain.from_iterable(dt['characters'].values)))
    gch[genre] = unique_characters
    print('%s: %d unique characters' % (genre, len(unique_characters)))

Action: 44462 unique characters
Adventure: 17355 unique characters
Animation: 5608 unique characters
Comedy: 60948 unique characters
Crime: 17425 unique characters
Documentary: 1981 unique characters
Drama: 79529 unique characters
Family: 3684 unique characters
Fantasy: 6621 unique characters
Foreign: 333 unique characters
History: 2621 unique characters
Horror: 18879 unique characters
Music: 3256 unique characters
Mystery: 5246 unique characters
Romance: 8840 unique characters
Science Fiction: 6203 unique characters
TV Movie: 2445 unique characters
Thriller: 12533 unique characters
War: 3446 unique characters
Western: 4748 unique characters


# Store processed data
Store keywords, and characters by gender as json file

In [20]:
r = json.dumps([gch, gkw])
# Dump data to a json file
with open('mvdb.json', 'w') as f:
    f.write(r)

mdb.to_csv('mvdb-table.csv', index=False)